In [2]:
# coding: utf-8 
import os
import numpy as np
import multiprocessing
from multiprocessing import Array
import time

In [3]:
def listdir_nohidden(path):
    return filter( lambda f: not f.startswith('.'), os.listdir(path))  

In [4]:
m_3d = Array('f', 256* 8* 1728)
# m[0] = a[0]
print type(m_3d)
print len(m_3d)
print m_3d

<class 'multiprocessing.sharedctypes.SynchronizedArray'>
3538944
<SynchronizedArray wrapper for <multiprocessing.sharedctypes.c_float_Array_3538944 object at 0x7fc8df99b710>>


In [4]:
filedir = '/media/jerry/Sexxy/aa-alexnet-tmp/log/'
trainlist_dir = '/home/jerry/data-alexnet/train_imglist.txt'
trainlist = np.loadtxt(trainlist_dir, dtype=str)
#获取当前文件夹中的文件名称列表  
dir_1=list(listdir_nohidden(filedir))
print dir_1

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '246', '247', '248', '249', '25', '250', '251', '252', '253', '254', '255', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '23', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '24', '240', '241', '242', '243', '244', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '13', '163', 

In [5]:
def worker(num1, num2, arr_3d):
    for j in range(num1, num2):
        print 'j: ', j
        kernel_num = int(dir_1[j])
        print 'Kernel or Log: ', kernel_num
        
        log_dir = filedir + dir_1[j] + '/kernel-' + dir_1[j] + '-log.txt'
#         print log_dir
        log = np.loadtxt(log_dir, dtype=np.str)
#         print log
        data_dir = '/media/jerry/Sexxy/aa-alexnet-tmp/data/' + dir_1[j] +'.txt'
        data = np.loadtxt(data_dir, dtype=np.float32)
        data = data.reshape(8, 1000, 1728)
#         print data.shape
#         print log_dir
#         print data_dir
        
        for i in range(8):
            img = log[2*i+1].split('/',8)[8]
            ##  /media/jerry/Data/ILSVRC2015/Data/CLS-LOC/train/n01664065/n01664065_18162.JPEG
            ## /run/user/1000/gvfs/smb-share:server=10.19.127.214,share=data/ILSVRC2015/Data/CLS-LOC/train/n03125729/n03125729_26786.JPEG
#             print img, len(img)
            if(len(img) > 35):
                img = log[2*i+1].split('/',10)[10]                
#                 print img
            itemindex = np.where(trainlist==img)
            print itemindex
            if (len(itemindex[0]) == 0):
                img = img.strip('G')
                itemindex = np.where(trainlist==img)
#                 print img
#                 print itemindex              



# #             else:
# #                 print 'itemindex is not none'
# #                 pass
                
#             print type(itemindex[0])
#             print len(itemindex[0])
#             print itemindex[1]
#             print len(itemindex)
#             print type(itemindex)

            num = itemindex[0][0]            
#             print type(num)
#             print num

            label = trainlist[num][1]
#             print type(label)
            label = int(label)
#             print label

            data_catch = data[i][label]
#             print data_catch.shape

            for k in range(1728):
                arr_3d[kernel_num*8*1728 + i*1728 + k] = data_catch[k]
        print 'j: ', j, 'Done'
 



def multi_process():
#     p1 = multiprocessing.Process(target = worker, args = (0,64,m_3d))
#     p2 = multiprocessing.Process(target = worker, args = (64,128,m_3d))
#     p3 = multiprocessing.Process(target = worker, args = (128,192,m_3d))
#     p4 = multiprocessing.Process(target = worker, args = (192,256,m_3d))

    p1 = multiprocessing.Process(target = worker, args = (0,42,m_3d))
    p2 = multiprocessing.Process(target = worker, args = (42,84,m_3d))
    p3 = multiprocessing.Process(target = worker, args = (84,126,m_3d))
    p4 = multiprocessing.Process(target = worker, args = (126,168,m_3d))
    p5 = multiprocessing.Process(target = worker, args = (168,210,m_3d))
    p6 = multiprocessing.Process(target = worker, args = (210,256,m_3d))   
    
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p6.start()
    
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    p5.join()
    p6.join()

#     p1 = multiprocessing.Process(target = worker, args = (195,196,m_3d))
#     p1.start()
#     p1.join()
    

# multi_process()


# conv5_coeff_3d = np.zeros(256* 8* 1728)

# for j in range(256 *8*1728):
#      conv5_coeff_3d[j] = m_3d[j]
# # print conv5_coeff_3d.reshape(256, 8, 1728)

# conv5_coeff_3d = conv5_coeff_3d.reshape(256, 8, 1728)

# np.save('/home/jerry/data-alexnet/npy/c5_256_8_1728.npy', conv5_coeff_3d)

# print 'All Process Done!'

In [6]:
multi_process()


conv5_coeff_3d = np.zeros(256* 8* 1728)

for j in range(256 *8*1728):
     conv5_coeff_3d[j] = m_3d[j]
# print conv5_coeff_3d.reshape(256, 8, 1728)

conv5_coeff_3d = conv5_coeff_3d.reshape(256, 8, 1728)

np.save('/home/jerry/data-alexnet/npy/c5_256_8_1728.npy', conv5_coeff_3d)

print 'All Process Done!'

j:  0
Kernel or Log:  0
j:  42
Kernel or Log:  252
j:  84
Kernel or Log:  237
j:  126
Kernel or Log:  172
j:  168
j:  210
Kernel or Log:  90
Kernel or Log:  50
(array([860432]), array([0]))
(array([565440]), array([0]))
(array([508895]), array([0]))
(array([538539]), array([0]))
(array([491531]), array([0]))
(array([425587]), array([0]))
(array([1257779]), array([0]))
(array([178330]), array([0]))
(array([1032715]), array([0]))
(array([1074680]), array([0]))
(array([1146216]), array([0]))
(array([468211]), array([0]))
(array([920650]), array([0]))
j:  42 Done
j:  43
(array([82410]), array([0]))
Kernel or Log:  253
(array([1080835]), array([0]))
(array([494786]), array([0]))
(array([737560]), array([0]))
(array([63769]), array([0]))
(array([890068]), array([0]))
(array([514969]), array([0]))
(array([1062892]), array([0]))
(array([875135]), array([0]))
(array([71810]), array([0]))
j:  210 Done
j:  211
Kernel or Log:  91
(array([456439]), array([0]))
j:  84 Done
j:  85
Kernel or Log:  238

(array([483656]), array([0]))
(array([1188582]), array([0]))
(array([243291]), array([0]))
j:  88 Done
j:  89
Kernel or Log:  241
(array([1035623]), array([0]))
(array([1246166]), array([0]))
(array([974407]), array([0]))
(array([384480]), array([0]))
(array([1198524]), array([0]))
(array([699483]), array([0]))
(array([309270]), array([0]))
(array([151650]), array([0]))
j:  130 Done
j:  131
Kernel or Log:  177
(array([894922]), array([0]))
(array([73126]), array([0]))
(array([396810]), array([0]))
(array([112712]), array([0]))
(array([391601]), array([0]))
(array([1147094]), array([0]))
(array([785346]), array([0]))
(array([669420]), array([0]))
j:  47 Done
j:  48
Kernel or Log:  28
(array([1261775]), array([0]))
(array([227764]), array([0]))
(array([613989]), array([0]))
(array([817552]), array([0]))
(array([270074]), array([0]))
(array([1000186]), array([0]))
(array([711969]), array([0]))
(array([986230]), array([0]))
j:  215 Done
j:  216
Kernel or Log:  96
(array([526169]), array([0

(array([596850]), array([0]))
j:  177 Done
j:  178
Kernel or Log:  60
(array([1118487]), array([0]))
(array([1146884]), array([0]))
(array([760597]), array([0]))
(array([372872]), array([0]))
(array([1253221]), array([0]))
(array([1176358]), array([0]))
(array([269718]), array([0]))
(array([212304]), array([0]))
j:  93 Done
j:  94
Kernel or Log:  191
(array([913171]), array([0]))
(array([663018]), array([0]))
(array([671405]), array([0]))
(array([231197]), array([0]))
(array([880419]), array([0]))
(array([1203722]), array([0]))
(array([11178]), array([0]))
(array([866721]), array([0]))
(array([318114]), array([0]))
(array([832820]), array([0]))
(array([750766]), array([0]))
j:  221 Done
j:  222
Kernel or Log:  132
(array([493597]), array([0]))
(array([161273]), array([0]))
(array([358679]), array([0]))
(array([527030]), array([0]))
(array([805442]), array([0]))
j:  52 Done
j:  53
Kernel or Log:  32
(array([541653]), array([0]))
(array([910851]), array([0]))
(array([687301]), array([0])

(array([1135984]), array([0]))
(array([725702]), array([0]))
(array([1196868]), array([0]))
(array([4215]), array([0]))
(array([647084]), array([0]))
(array([620988]), array([0]))
(array([113072]), array([0]))
(array([158034]), array([0]))
j:  139 Done
j:  140
Kernel or Log:  185
(array([576957]), array([0]))
(array([857489]), array([0]))
(array([687759]), array([0]))
(array([389681]), array([0]))
(array([730460]), array([0]))
(array([281965]), array([0]))
(array([485474]), array([0]))
(array([1095181]), array([0]))
j:  14 Done
j:  15
Kernel or Log:  111
(array([500601]), array([0]))
(array([1013589]), array([0]))
(array([272585]), array([0]))
(array([21602]), array([0]))
(array([274244]), array([0]))
(array([361852]), array([0]))
(array([561715]), array([0]))
(array([595345]), array([0]))
j:  98 Done
j:  99
Kernel or Log:  196
(array([1261983]), array([0]))
(array([1100820]), array([0]))
(array([891152]), array([0]))
(array([1001554]), array([0]))
(array([1047960]), array([0]))
(array

(array([491174]), array([0]))
(array([35497]), array([0]))
(array([1176705]), array([0]))
(array([643523]), array([0]))
(array([818655]), array([0]))
(array([1180568]), array([0]))
j:  187 Done
j:  188
Kernel or Log:  7
(array([506183]), array([0]))
(array([707511]), array([0]))
(array([745023]), array([0]))
(array([1028016]), array([0]))
(array([35187]), array([0]))
(array([218717]), array([0]))
(array([924874]), array([0]))
(array([739872]), array([0]))
j:  19 Done
j:  20
Kernel or Log:  116
(array([301892]), array([0]))
(array([976598]), array([0]))
(array([991628]), array([0]))
(array([124957]), array([0]))
(array([828422]), array([0]))
(array([720460]), array([0]))
(array([876325]), array([0]))
(array([704709]), array([0]))
j:  102 Done
j:  103
Kernel or Log:  2
(array([1073666]), array([0]))
(array([828341]), array([0]))
(array([1043685]), array([0]))
(array([775684]), array([0]))
(array([1023926]), array([0]))
(array([1279172]), array([0]))
(array([134319]), array([0]))
(array([

(array([1184423]), array([0]))
(array([121163]), array([0]))
(array([168374]), array([0]))
(array([593988]), array([0]))
(array([591928]), array([0]))
j:  67 Done
j:  68
Kernel or Log:  222
(array([183597]), array([0]))
(array([839929]), array([0]))
(array([643301]), array([0]))
(array([720594]), array([0]))
(array([429341]), array([0]))
(array([633887]), array([0]))
(array([311098]), array([0]))
(array([40130]), array([0]))
j:  24 Done
j:  25
Kernel or Log:  120
(array([663337]), array([0]))
(array([22443]), array([0]))
(array([818661]), array([0]))
(array([311614]), array([0]))
(array([574919]), array([0]))
(array([478751]), array([0]))
(array([17101]), array([0]))
(array([1113631]), array([0]))
j:  192 Done
j:  193
Kernel or Log:  74
(array([844349]), array([0]))
(array([918754]), array([0]))
(array([86043]), array([0]))
(array([563115]), array([0]))
(array([480513]), array([0]))
(array([1195336]), array([0]))
(array([620556]), array([0]))
(array([629522]), array([0]))
j:  107 Done


(array([216246]), array([0]))
(array([305928]), array([0]))
(array([851380]), array([0]))
j:  111 Done
j:  112
Kernel or Log:  207
(array([485891]), array([0]))
(array([885202]), array([0]))
(array([1233176]), array([0]))
(array([919850]), array([0]))
(array([908479]), array([0]))
(array([715311]), array([0]))
(array([780626]), array([0]))
(array([647939]), array([0]))
j:  197 Done
j:  198
Kernel or Log:  8
(array([931572]), array([0]))
(array([1168517]), array([0]))
(array([1044012]), array([0]))
(array([362270]), array([0]))
(array([303192]), array([0]))
(array([459407]), array([0]))
(array([1101321]), array([0]))
(array([748306]), array([0]))
j:  154 Done
j:  155
Kernel or Log:  39
(array([211399]), array([0]))
(array([417550]), array([0]))
(array([1062085]), array([0]))
(array([871588]), array([0]))
(array([456488]), array([0]))
(array([618342]), array([0]))
(array([740094]), array([0]))
(array([1064939]), array([0]))
j:  240 Done
j:  241
Kernel or Log:  15
(array([484999]), array(

(array([699842]), array([0]))
(array([175831]), array([0]))
j:  116 Done
j:  117
Kernel or Log:  164
(array([878604]), array([0]))
(array([1226087]), array([0]))
(array([447580]), array([0]))
(array([1007539]), array([0]))
(array([847641]), array([0]))
(array([488729]), array([0]))
(array([766464]), array([0]))
(array([1177218]), array([0]))
j:  33 Done
j:  34
Kernel or Log:  129
(array([997618]), array([0]))
(array([1134025]), array([0]))
(array([586459]), array([0]))
(array([732723]), array([0]))
(array([509033]), array([0]))
(array([365828]), array([0]))
(array([1114423]), array([0]))
(array([423824]), array([0]))
j:  202 Done
j:  203
Kernel or Log:  84
(array([1230344]), array([0]))
(array([430323]), array([0]))
(array([824161]), array([0]))
(array([357990]), array([0]))
(array([1192544]), array([0]))
(array([803381]), array([0]))
(array([38921]), array([0]))
(array([92795]), array([0]))
j:  78 Done
j:  79
Kernel or Log:  232
(array([89663]), array([0]))
(array([19397]), array([0])

(array([440799]), array([0]))
j:  163 Done
j:  164
Kernel or Log:  47
(array([887276]), array([0]))
(array([778635]), array([0]))
(array([1258883]), array([0]))
(array([921446]), array([0]))
(array([176864]), array([0]))
(array([246440]), array([0]))
(array([124424]), array([0]))
(array([1258244]), array([0]))
j:  38 Done
j:  39
Kernel or Log:  25
(array([1004356]), array([0]))
(array([1156528]), array([0]))
(array([1219158]), array([0]))
(array([1215397]), array([0]))
(array([323844]), array([0]))
(array([1225203]), array([0]))
(array([752680]), array([0]))
(array([1167200]), array([0]))
j:  121 Done
j:  122
Kernel or Log:  169
(array([1054519]), array([0]))
(array([147307]), array([0]))
(array([1163737]), array([0]))
(array([553019]), array([0]))
(array([1222339]), array([0]))
(array([779005]), array([0]))
(array([453958]), array([0]))
(array([279987]), array([0]))
j:  207 Done
j:  208
Kernel or Log:  89
(array([1205557]), array([0]))
(array([932015]), array([0]))
(array([1091967]), 

In [ ]:
## Test Code

filedir = '/media/jerry/Sexxy/aa-alexnet-tmp/log/'
#获取当前文件夹中的文件名称列表  
dir_1=list(listdir_nohidden(filedir))
print dir_1
trainlist_dir = '/home/jerry/data-alexnet/train_imglist.txt'





In [ ]:
i = 195

log_dir = filedir + dir_1[i] + '/kernel-' + dir_1[i] + '-log.txt'
print log_dir

# data_dir = '/media/jerry/Sexxy/aa-alexnet-tmp/data/' + dir_1[i] +'.txt'
# print data_dir

# data = np.loadtxt(data_dir,dtype=np.float32)
# trainlist = np.loadtxt(trainlist_dir, dtype=str)
# trainlist = np.loadtxt(trainlist_dir, dtype='|S30')
# print data.shape
# data = data.reshape(8, 1000, 1728)
# print data.shape

In [ ]:
# img ='n03899768/n03899768_125766.JPEG'
# print len(img)
# img = img.strip('G')
# print img
# itemindex = np.where(trainlist==img)
# print itemindex
# if (len(itemindex) == 0 ):
#     pass
# #     img = img[]

In [ ]:
trainlist[905647]

In [ ]:
data[0][1]

a = np.loadtxt(log_dir, dtype=np.str)
trainlist = np.loadtxt(trainlist_dir, dtype=str)
print a



print len(a)
print len(trainlist)
print trainlist[0][1]

for i in range(8):
    img = a[2*i+1].split('/',8)[8]
    path = a[2*i+2]
#     print path
#     print img
#     if img in trainlist: print 1
    itemindex = np.where(trainlist==img)
#     print itemindex
#     print type(itemindex)
    num = itemindex[0][0]
#     print type(num)
#     print num
    label = trainlist[num][1]
    print type(label)
    label = int(label)
    print label

    data = np.loadtxt(path, dtype=np.float32)
    data = data.reshape(1000, 1728)
    print data.shape
    print data[label]/0.00001
    
    
#     print a[2*i+2]
#     print '###'